In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Esophageal_Cancer/GSE100843'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression data from nonrandomized trial of vitamin D in Barrett's esophagus"
!Series_summary	"Vitamin D deficiency has been associated with increased esophageal cancer risk.  Vitamin D controls many downstream regulators of cellular processes including proliferation, apoptosis, and differentiation.  We evaluated the effects of vitamin D supplementation on global gene expression in patients with Barrett's esophagus."
!Series_summary	"We used microarrays to assess global gene expression in Barrett's esophagus patients who received vitamin D supplementation."
!Series_overall_design	"Patients in Arm A with Barrett's esophagus with high grade dysplasia were given vitamin D3 50,000 IU weekly for 2 weeks.  Patients in Arm B with Barrett's esophagus with low grade dysplasia or no dysplasia were given vitamin D3 50,000 IU weekly for 12 weeks.  In both arms, biopsies were obtained from two sites: Barrett's esophagus segment (IM) and normal squamous mucosa 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Step 1: Check for gene expression data availability
is_gene_available = True  # Based on the background info, gene expression data seems present

# Step 2: Check variable availability and data type conversion
# Esophageal_Cancer, age, gender
# Since the sample characteristics dictionary does not provide clear keys for our variables, set to None.

# Data type conversion functions
def convert_trait(val):
    try:
        value = val.split(":")[1].strip()
        if value == "Esophageal Cancer":
            return 1
        elif value == "No Esophageal Cancer":
            return 0
        else:
            return None
    except:
        return None

def convert_age(val):
    try:
        return int(val.split(":")[1].strip())
    except:
        return None

def convert_gender(val):
    try:
        value = val.split(":")[1].strip().lower()
        if value == "male":
            return 1
        elif value == "female":
            return 0
        else:
            return None
    except:
        return None 

# Step 3: Save cohort information
save_cohort_info('GSE100843', './preprocessed/Esophageal_Cancer/cohort_info.json', is_gene_available, trait_row is not None)

# Step 4: Clinical Feature Extraction
# Since trait_row is None, skip this step
